In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
pd.set_option('display.max_columns', None)

In [7]:
ruta3 = r'https://github.com/gmarinr/TOPD/raw/refs/heads/main/Archivos/GRD_Completo.parquet'
df_GRD = pd.read_parquet(ruta3)
df_GRD.head(2)

,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,SERVICIO_SALUD,TIPO_INGRESO,FECHA_INGRESO,FECHAALTA,TIPOALTA,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,EDAD,DIAS_ESTANCIA,HOSPITAL,DESCCRIPCION_GRD,Versión,Descripción,Categoría,Sección,Capítulo,NEXT_FECHA_INGRESO,DIAS_REINGRESO,READM_7D,READM_30D,READM_60D,READM_90D,PROCEDIMIENTO1,PROCEDIMIENTO2,PROCEDIMIENTO3,PROCEDIMIENTO4,PROCEDIMIENTO5,PROCEDIMIENTO6,PROCEDIMIENTO7,PROCEDIMIENTO8,PROCEDIMIENTO9,PROCEDIMIENTO10,PROCEDIMIENTO11,PROCEDIMIENTO12,PROCEDIMIENTO13,PROCEDIMIENTO14,PROCEDIMIENTO15,PROCEDIMIENTO16,PROCEDIMIENTO17,PROCEDIMIENTO18,PROCEDIMIENTO19,PROCEDIMIENTO20,PROCEDIMIENTO21,PROCEDIMIENTO22,PROCEDIMIENTO23,PROCEDIMIENTO24,PROCEDIMIENTO25,PROCEDIMIENTO26,PROCEDIMIENTO27,PROCEDIMIENTO28,PROCEDIMIENTO29,PROCEDIMIENTO30
0,66988070.0,HOMBRE,1977-07-12,METROPOLITANO SURORIENTE,URGENCIA,2023-03-29,2023-04-13,DOMICILIO,1.1263,3.0,2.0,46.0,15,Complejo Hospitalario Dr. Sótero del Río (San...,"MH DISFUNCIÓN, REACCIÓN O COMPL. POR DISP., I...",CIE-v2013,Falla y rechazo de trasplante de riñón,T86 FALLA Y RECHAZO DEL TRASPLANTE DE ÓRGANOS ...,T80-T88 COMPLICACIONES DE LA ATENCIÓN QUIRÚRG...,"Cap.19 LESIONES TRAUMÁTICAS, ENVENENAMIENTOS ...",2023-06-22,70.0,0,0,0,1,5523.0,8703.0,8744.0,8952.0,8875.0,9039.0,9059.0,8929.0,9132.0,9139.0,8906.0,8913.0,9318.0,68.0,3891.0,8966.0,99.21,99.19,99.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66988070.0,HOMBRE,1977-07-12,METROPOLITANO SURORIENTE,URGENCIA,2023-03-29,2023-04-13,DOMICILIO,1.1263,3.0,2.0,46.0,15,Complejo Hospitalario Dr. Sótero del Río (San...,"MH DISFUNCIÓN, REACCIÓN O COMPL. POR DISP., I...",CIE-v2013,Falla y rechazo de trasplante de riñón,T86 FALLA Y RECHAZO DEL TRASPLANTE DE ÓRGANOS ...,T80-T88 COMPLICACIONES DE LA ATENCIÓN QUIRÚRG...,"Cap.19 LESIONES TRAUMÁTICAS, ENVENENAMIENTOS ...",2023-06-22,70.0,0,0,0,1,9671.0,9604.0,9396.0,9059.0,8966.0,8929.0,9132.0,8875.0,9039.0,9312.0,9318.0,9394.0,9919.0,966.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
